## This Code Tries to Retrieve Information from the Patio de Comedias' Website 

First, we import all the libraries to ensure we can use different features in this process

In [10]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup #BeautifulSoup for HTML Scrapping
import collections
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium imported')
print('Pandas imported')
print('BeautifulSoup imported')

Folium imported
Pandas imported
BeautifulSoup imported


Now, we read get the page source from the website of interest

In [11]:
#To feed the webpage directly we can do the following:
source=requests.get('http://patiodecomedias.org/').text
newsoup = BeautifulSoup(source, 'lxml') #create a new beautiful soup object
#print(newsoup.prettify())

In [12]:
figure = newsoup.find_all('figure') #find the table body element where all the information is embedded
#<figure class="aligncenter"> this element contains the info for each event entry
figure[0] #this is the first entry in the website correspongind to "la reina de la china"

<figure class="aligncenter"><a href="http://patiodecomedias.org/la-reina-de-la-china/"><img alt="" class="wp-image-1033" sizes="(max-width: 1024px) 100vw, 1024px" src="http://patiodecomedias.org/wp-content/uploads/2019/05/La-reina-de-la-China-0619-1024x625.jpg" srcset="http://patiodecomedias.org/wp-content/uploads/2019/05/La-reina-de-la-China-0619-1024x625.jpg 1024w, http://patiodecomedias.org/wp-content/uploads/2019/05/La-reina-de-la-China-0619-300x183.jpg 300w, http://patiodecomedias.org/wp-content/uploads/2019/05/La-reina-de-la-China-0619-768x469.jpg 768w, http://patiodecomedias.org/wp-content/uploads/2019/05/La-reina-de-la-China-0619.jpg 1110w"/></a></figure>

Now we need to access the anchor tag with the corresponding url so that we can go deeper and retrieve information regarding the event

In [65]:
link=figure[0].find('a', href=True) #from the first figure section, fint the anchor tag that has an href class
url= link['href'] #then we extract the url only
#print(url) #print the url :)
URLS=[]
URLS.append(url)
#print(URLS)

### Accessing the url from the first figure element 

We proceed in a similar manner to the first section, where we create a BeautifulSoup object and look for a specific tag or element in the source code

In [17]:
source_2=requests.get(str(url)).text
newsoup_2 = BeautifulSoup(source_2, 'lxml') #create a new beautiful soup object
print(newsoup_2.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="http://patiodecomedias.org/xmlrpc.php" rel="pingback"/>
  <title>
   La Reina de la China
  </title>
  <link href="//fonts.googleapis.com" rel="dns-prefetch"/>
  <link href="//s.w.org" rel="dns-prefetch"/>
  <link href="http://patiodecomedias.org/feed/" rel="alternate" title=" » Feed" type="application/rss+xml"/>
  <link href="http://patiodecomedias.org/comments/feed/" rel="alternate" title=" » RSS de los comentarios" type="application/rss+xml"/>
  <!-- This site uses the Google Analytics by MonsterInsights plugin v7.6.0 - Using Analytics tracking - https://www.monsterinsights.com/ -->
  <!-- Nota: MonsterInsights no está actualmente configurado en este sitio. El dueño del sitio necesita identificarse usando su cuenta de Google Analytics en el panel de ajustes de Monst

From the BeautifulSoup object we can extract the title of the event as well as the times, dates and fees

In [57]:
title = newsoup_2.find('title')
title.text #this is the text embedded in the title tag
titulos=[] #initialize an empty list that will be used later on
titulos.append(title.text) #append the title to the titles' list
titulos #display the titles' list

['La Reina de la China']

In [58]:
p_center=newsoup_2.find_all('p',{"style" : "text-align:center"}) #create a list that contains the info for the p tags with centered text
p_center #list containing 3 items

[<p style="text-align:center"><strong>FUNCIONES: DEL MIÉRCOLES 19 DE JUNIO AL DOMINGO 28 DE JULIO</strong></p>,
 <p style="text-align:center"><strong>HORARIOS:  JUEVES, VIERNES Y SÁBADO LAS 20H30; DOMINGO 18H30</strong></p>,
 <p style="text-align:center"><strong>ENTRADAS: $20 General, y $10 Estudiantes, Tercera Edad y Personas con discapacidad.</strong></p>]

In [75]:
funciones=[] #empty list to store event dates
funcion = p_center[0].strong.text
funcion = funcion.replace('\xa0','')
funcion = funcion.replace('FUNCIONES:','')
funciones.append(funcion)
#print(funciones)

In [84]:
horarios=[] #empty list of times
hora= p_center[1].strong.text
hora= hora.replace('\xa0','')
hora = hora.replace('HORARIOS:','')
horarios.append(hora)
#print(horarios)

In [85]:
entradas=[]
entr=p_center[2].strong.text
entr=entr.replace('\xa0','')
entr = entr.replace('ENTRADAS:','')
entr = entr.strip()
entradas.append(entr)
#print(entradas) #uncomment this line to look at the way the string has been written down. There is a problem with '$' sign

#entr2= '20 General, y 10 Estudiantes, Tercera Edad y Personas con discapacidad'
#entradas.append(entr2)

In [87]:
info={'Obra':titulos,'Link': URLS,'Funciones':funciones,'Horarios':horarios,'Entradas':entradas}
df=pd.DataFrame(info)
df.head()

,Obra,Link,Funciones,Horarios,Entradas
0,La Reina de la China,http://patiodecomedias.org/la-reina-de-la-china/,DEL MIÉRCOLES 19 DE JUNIO AL DOMINGO 28 DE JULIO,"JUEVES, VIERNES Y SÁBADO LAS 20H30; DOMINGO 18H30","$20 General, y $10 Estudiantes, Tercera Edad y..."
